<h2 align='center'>| CodeClause Data Science Internship |</h2>
<h3 align='right'>-by D VAMSIDHAR</h3>


<h2 align='center'>Credit Card Fraud Detection</h2>

In [34]:
#importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings

warnings.filterwarnings('ignore')

In [2]:
#importing data into the workspacing
data = pd.read_csv('creditcard.csv')
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [4]:
#checking for null values in the dataset
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
class     0
dtype: int64

### No null values exist in the dataset

In [6]:
#distribution of legit transactions and fraud transaction
data['class'].value_counts()

0    284315
1       492
Name: class, dtype: int64

### This is a highly unbalanced dataset

In [9]:
#seperating the data for analysis
legit = data[data['class'] == 0]
fraud = data[data['class'] == 1]

In [10]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [11]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [12]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [14]:
data.groupby('class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Since the above dataset is unbalanced we need to create a new dataset with sample legit dataset with similar distribution of that of fraud distribution.

In [15]:
legit_sample = legit.sample(n=492)

In [16]:
#concatinating the dataframes
new_sample_data = pd.concat([legit_sample, fraud], axis=0)

In [17]:
new_sample_data.to_csv('New Sample Data.csv')

In [18]:
new_sample_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,class
99789,67281.0,-0.892212,0.422794,0.098428,-0.550371,-3.485293,2.164113,-1.561919,-6.209458,0.949982,...,-1.849651,0.830185,-0.225534,0.164057,0.121370,0.259101,0.536426,0.159468,500.00,0
76510,56604.0,0.739033,-0.561333,1.085235,1.082975,-0.163843,2.208533,-0.840440,0.786369,0.531866,...,0.172098,0.622576,0.020591,-0.990243,0.051309,-0.309726,0.125081,0.031042,105.00,0
266475,162344.0,-0.663494,0.349452,0.464813,-1.772132,1.813338,4.530121,-0.694751,0.420088,0.475909,...,0.880213,0.095339,-0.002108,0.704620,-0.300117,0.048461,0.232474,0.136131,47.50,0
243186,151834.0,1.893685,-0.367573,-0.805155,0.492820,-0.384896,-0.803834,-0.053714,-0.234883,0.979957,...,0.077723,0.323212,0.105491,-0.061675,-0.063232,-0.339552,0.007818,-0.033755,68.18,0
13432,23765.0,-6.257268,-4.799119,-0.389479,1.314958,0.411875,0.355027,1.925089,-0.169486,1.689391,...,-1.158800,-0.720885,2.736688,-1.373999,1.155501,0.381775,1.114491,-0.274602,683.34,0


In [20]:
new_sample_data['class'].value_counts()

0    492
1    492
Name: class, dtype: int64

## Splitting the dataset into independent and dependent variables

In [25]:
X = new_sample_data.drop(columns='class', axis=1)
Y = new_sample_data['class']

In [26]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=2)

In [27]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


In [47]:
class_names = new_sample_data['class'].unique()
class_names

array([0, 1], dtype=int64)

## Prediction using Logitic Regression

In [28]:
logistic_model = LogisticRegression()

In [29]:
logistic_model.fit(X_train, Y_train)

LogisticRegression()

In [53]:
X_train_prediction = logistic_model.predict(X_train)
training_accuracy = accuracy_score(X_train_prediction, Y_train)
print(training_accuracy*100,'%')

92.24904701397712 %


In [54]:
X_test_prediction = logistic_model.predict(X_test)
testing_accuracy = accuracy_score(X_test_prediction, Y_test)
print(testing_accuracy*100,'%')

90.35532994923858 %


## Prediction using Random Forest classifier

In [49]:
random_forest_model = RandomForestClassifier()

In [50]:
random_forest_model.fit(X_train, Y_train)

RandomForestClassifier()

In [52]:
X_train_prediction = random_forest_model.predict(X_train)
training_accuracy = accuracy_score(X_train_prediction, Y_train)
print(training_accuracy*100,'%')

100.0 %


In [56]:
X_test_prediction = random_forest_model.predict(X_test)
testing_accuracy = accuracy_score(X_test_prediction, Y_test)
print(testing_accuracy*100,'%')

94.41624365482234 %


## Prediction using Decision Tree Classifier

In [57]:
decision_tree_model = DecisionTreeClassifier()

In [58]:
decision_tree_model.fit(X_train, Y_train)

DecisionTreeClassifier()

In [59]:
X_train_prediction = decision_tree_model.predict(X_train)
training_accuracy = accuracy_score(X_train_prediction, Y_train)
print(training_accuracy*100,'%')

100.0 %


In [60]:
X_test_prediction = decision_tree_model.predict(X_test)
testing_accuracy = accuracy_score(X_test_prediction, Y_test)
print(testing_accuracy*100,'%')

91.37055837563452 %


## It can be clearly observed that out of three classification algorithms used Random Forest Classifier has been proven to be highly efficient.

In [61]:
#prediction system
#let's test by giving the input by ourselves
input_data = (0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = random_forest_model.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
    print('The transaction is not fraudelent')
else:
    print('The transaction is fraudalent')

[0]
The transaction is not fraudelent


In [62]:
input_data = (406,-2.3122265423263,1.95199201064158,-1.60985073229769,3.9979055875468,-0.522187864667764,-1.42654531920595,-2.53738730624579,1.39165724829804,-2.77008927719433,-2.77227214465915,3.20203320709635,-2.89990738849473,-0.595221881324605,-4.28925378244217,0.389724120274487,-1.14074717980657,-2.83005567450437,-0.0168224681808257,0.416955705037907,0.126910559061474,0.517232370861764,-0.0350493686052974,-0.465211076182388,0.320198198514526,0.0445191674731724,0.177839798284401,0.261145002567677,-0.143275874698919,0)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = random_forest_model.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
    print('The transaction is not fraudelent')
else:
    print('The transaction is fraudalent')

[1]
The transaction is fraudalent
